# Predictions
This is our code for taking in a user's text and making a prediction based on that. 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# set configurations
pd.set_option('display.max_columns', 100)
sns.set_style("white")

# model imports
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
import pickle
import joblib

# NLP Imports
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

from PIL import Image
import wordninja

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ayaanhaque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ayaanhaque/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def processing_text(series_to_process):
    new_list = []
    tokenizer = RegexpTokenizer(r'(\w+)')
    lemmatizer = WordNetLemmatizer()

    for i in range(len(series_to_process)):
        # tokenized item in a new list
        dirty_string = (series_to_process)[i].lower()
        words_only = tokenizer.tokenize(dirty_string) # words_only is a list of only the words, no punctuation
        #Lemmatize the words_only
        words_only_lem = [lemmatizer.lemmatize(i) for i in words_only]
        # removing stop words
        words_without_stop = [i for i in words_only_lem if i not in stopwords.words("english")]
        # return seperated words
        long_string_clean = " ".join(word for word in words_without_stop)
        new_list.append(long_string_clean)
        return new_list

In [3]:
model = joblib.load("modelClusteredLabels.h5")

In [5]:
# using the function for out two narrowed models

# models
steps_titles = ['hvec + multi_nb(tuning)','tvec + multi_nb(tuning)']

# pipelines
steps_list = [ 
    [('hv', HashingVectorizer(alternate_sign=False)),('multi_nb', MultinomialNB())],
    [('tv', TfidfVectorizer()),('multi_nb', MultinomialNB())]
]

# parameters
pipe_params = [
    {'hv__stop_words':['english'], 'hv__ngram_range':[(1,1),(1,2),(1,3)], 'hv__n_features': [50, 150, 300, 500, 800, 1000]},
    {'tv__stop_words':['english'], 'tv__ngram_range':[(1,1),(1,2),(1,3)],'tv__max_features': [20, 30, 50, 54, 100],'tv__min_df': [2, 3],'tv__max_df': [.2, .25, .3, .35,.4]},
]   

# gridsearch_multi(steps_titles, steps_list, pipe_params)

In [6]:
text = "Hello. Depression has always been a secondary problem for me, with my main antagonist being severe Harm OCD. But since my relationship ended 8 months ago, I've been stuck in this horrific cycle of absolutely loathing myself, feeling heavy/tired and totally unmotivated to do anything. It's like I'm living in a 2 dimensional world. Nothing in life jumps out and catches my attention like it used to. I used to be quite creative but it's just taken a nose dive. Any work I do is utterly awful and I'm amazed I'm not been kicked off projects (I work freelance). I wake up and I just want to be dead, quite honestly. In fact in the last few weeks I've even found getting out of bed to be a monumental struggle in itself, where I'm almost in tears from the weight of everything."
# text2 = "Hello. Depression has always been a secondary problem for me, with my main antagonist being severe Harm OCD. But since my relationship ended 8 months ago, I've been stuck in this horrific cycle of absolutely loathing myself, feeling heavy/tired and totally unmotivated to do anything. It's like I'm living in a 2 dimensional world. Nothing in life jumps out and catches my attention like it used to. I used to be quite creative but it's just taken a nose dive. Any work I do is utterly awful and I'm amazed I'm not been kicked off projects (I work freelance). I wake up and I just want to be dead, quite honestly. In fact in the last few weeks I've even found getting out of bed to be a monumental struggle in itself, where I'm almost in tears from the weight of everything."
# text = "why does it hurt so much? Why can’t I be happy without it? There’s this empty void in my heart that gets bigger everyday. I’m just waiting until it eats me up, since I’ll never have 2 sided love."
# text = "I don't know what I'm gonna do if I don't perform the way I should. But I certainly know that I will die if he does. In case everything goes wrong, I'll leave. There's no other way. The day of my death is nearing. It's just 3 months away. Love you all. Have a good day."
text_array = pd.Series(text)
print(type(text_array))
print(text_array)
processed_text = processing_text(text_array)

processed_array = pd.Series(processed_text)

print(processed_array)

<class 'pandas.core.series.Series'>
0    Hello. Depression has always been a secondary ...
dtype: object
0    hello depression ha always secondary problem m...
dtype: object


In [10]:
tvec_optimised = TfidfVectorizer(max_df= 0.5, min_df=2, ngram_range=(1, 3),stop_words = 'english')
processed_text_tvec = tvec_optimised.fit_transform(processed_array).todense()

prediction = model.predict(processed_text_tvec)
print(prediction[0])

ValueError: max_df corresponds to < documents than min_df

This is our prediction model. We first convert the inputted text into an array. This array is then tokenized and lemmatized for our model. Finally, the concated text is converted into a vector and inputted into our preloaded model and received a prediction.